# Recommendation System Item Based Collaborative Filtering

In [1]:
! pip install surprise
! pip install nlp-id

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 5.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633975 sha256=5d73c535f7eb4988086ab179c6e33d5da82595dc4156076ec83adbfb4304a68a
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.9 MB 8.8 MB/s 
     |████████████████████████████████| 7.0 MB 39.1 MB/s 
     |████████████████████████████████| 1.5 MB 26.4 MB/s 
  Created wheel for nlp-id: filename=nlp_id-0.1.12.0-py3-none-any.whl size=8074104 sha256=22494a39f90d53236c551326da4443007af223331360eb6387781045a63dbbe9
  Stored in directory: /root/.cache/pip/wheels/b2/50/48/da59531125bd94f48dfe66140f41d8fd8a4f0

Library Import

In [4]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from nlp_id.tokenizer import Tokenizer
from nlp_id.lemmatizer import Lemmatizer
from nltk.corpus import stopwords
from nltk.corpus import stopwords

import nltk
import surprise

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Loading

Tourism Description dataset.

In [5]:
dataset_path = "tourism_with_id.csv"
data_df = pd.read_csv(dataset_path, delimiter = ',')

data_df.drop(["Unnamed: 11", "Unnamed: 12"], axis = 1, inplace = True)
data_df.head(5)

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134


In [6]:
data_df.describe()

,Place_Id,Price,Rating,Time_Minutes,Lat,Long
count,437.000000,437.000000,437.000000,205.000000,437.000000,437.000000
mean,219.000000,24652.173913,4.442792,82.609756,-7.095438,109.160142
std,126.295289,66446.374709,0.208587,52.872339,0.727241,1.962848
min,1.000000,0.000000,3.400000,10.000000,-8.197894,103.931398
25%,110.000000,0.000000,4.300000,45.000000,-7.749590,107.578369
50%,219.000000,5000.000000,4.500000,60.000000,-7.020524,110.237468
75%,328.000000,20000.000000,4.600000,120.000000,-6.829411,110.431869
max,437.000000,900000.000000,5.000000,360.000000,1.078880,112.821662


Rating Dataset

In [7]:
rating_dataset_path = "tourism_rating.csv"
rating_data_df = pd.read_csv(rating_dataset_path)

rating_data_df.head(5)

,User_Id,Place_Id,Place_Ratings
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4


Attribute Variables

In [8]:
description_field = "Description"
category_field = "Category"
tokenized_description_field = "Tokenized_Description"

## Data Preprocessing

Lemmatize Text.

In [9]:
indo_lemmatizer = Lemmatizer()

index = 0

tokenized_descriptions = []

for index, row in data_df.iterrows():
  description = row[description_field]
  tokenized_sentence = indo_lemmatizer.lemmatize(description)
  tokenized_descriptions.append(tokenized_sentence)

data_df[tokenized_description_field] = tokenized_descriptions
data_df.head(5)

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Tokenized_Description
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,monumen nasional atau yang populer singkat den...
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,kota tua di jakarta yang juga nama kota tua pu...
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,dunia fantasi atau sebut juga dufan adalah tem...
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,taman mini indonesia indah rupa suatu kawasan ...
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,atlantis water adventure atau kenal dengan atl...


## Data Modelling

Make a Sample Data

In [44]:
sample_description = "Dufan, Payung, dan Seni Rupa"
sample_categories = "Taman Hiburan"

Make Machine Learning Model (KNN Recommendation System).

In [45]:
def get_categories(sample_category):
  dataset = data_df
  categories = dataset[category_field]

  count_vectorizer = CountVectorizer()
  vector_components = count_vectorizer.fit_transform(categories)
  sample_category_vector_component = count_vectorizer.transform([sample_category])
  index_to_word_mapping = count_vectorizer.get_feature_names()
  return vector_components, sample_category_vector_component, index_to_word_mapping

def get_descriptions(sample_description):
  dataset = data_df
  descriptions = dataset[description_field]
  
  indonesian_stopwords = stopwords.words("indonesian")
  tf_idf_vectorizer = TfidfVectorizer(stop_words = indonesian_stopwords)
  vector_components = tf_idf_vectorizer.fit_transform(descriptions)
  sample_description_vector_component = tf_idf_vectorizer.transform([sample_description])
  index_to_word_mapping = tf_idf_vectorizer.get_feature_names()
  
  return vector_components, sample_description_vector_component, index_to_word_mapping

category_vector_components, sample_category_vector_component, category_index_to_word_mapping = get_categories("Museum")
description_vector_components, sample_description_vector_component, description_index_to_word_mapping = get_descriptions("Replika")

print(category_index_to_word_mapping[:10])
print(description_index_to_word_mapping[:10])

['alam', 'bahari', 'budaya', 'cagar', 'hiburan', 'ibadah', 'perbelanjaan', 'pusat', 'taman', 'tempat']
['00', '000', '01', '016', '02', '02223', '023', '039', '04', '05']


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Use K Nearest Neighbor to Make Content Based Recommendation based on Descriptions and Categories.

In [46]:
def recommend_travelling_places_using_knn(sample_description):
  dataset = data_df
  description_vector_components, sample_description_vector_component, index_to_word_mapping = get_descriptions(sample_description)

  nearest_neighbors = NearestNeighbors(n_neighbors = 10)
  nearest_neighbors.fit(description_vector_components)
  k_nearest_neighbors_scores = nearest_neighbors.kneighbors(sample_description_vector_component)
  
  return k_nearest_neighbors_scores

top_n_distances, top_n_indexes_ranking = recommend_travelling_places_using_knn(sample_description)

print(top_n_distances)
print(top_n_indexes_ranking.flatten())

[[1.14236799 1.1768974  1.20301053 1.20945773 1.2764293  1.2835308
  1.29355222 1.29587453 1.29747272 1.31983445]]
[  2 145  41  39 166 271 353  38  14 240]


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [42]:
def get_top_n_recommendations_based_on_similarity_scores(df, top_n_indexes):
  top_n_df = df.iloc[top_n_indexes]
  return top_n_df

get_top_n_recommendations_based_on_similarity_scores(data_df, top_n_indexes_ranking.flatten())

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Tokenized_Description
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,dunia fantasi atau sebut juga dufan adalah tem...
145,146,Bukit Wisata Pulepayung,Pule Payung Yogyakarta. Sebuah objek wisata po...,Cagar Alam,Yogyakarta,10000,4.5,NaN,"{'lat': -7.800110599999999, 'lng': 110.1238954}",-7.800111,110.123895,pule payung yogyakarta buah objek wisata popul...
166,167,Geoforest Watu Payung Turunan,Bagi para pemburu keindahan matahari terbit ya...,Cagar Alam,Yogyakarta,0,4.5,60.0,"{'lat': -7.974294, 'lng': 110.4364363}",-7.974294,110.436436,bagi para buru indah matahari terbit yang ting...
23,24,Museum Nasional,Museum Nasional Republik Indonesia atau Museum...,Budaya,Jakarta,5000,4.6,NaN,"{'lat': -6.176402099999999, 'lng': 106.8215901}",-6.176402,106.821590,museum nasional republik indonesia atau museum...
41,42,Museum Seni Rupa dan Kramik,Museum Seni Rupa dan Keramik (Indonesia: Museu...,Budaya,Jakarta,5000,4.4,NaN,"{'lat': -6.1342598, 'lng': 106.814552}",-6.134260,106.814552,museum seni rupa dan keramik indonesia museum ...
73,74,Museum Tengah Kebun,Museum di Tengah Kebun adalah sebuah museum pe...,Budaya,Jakarta,0,4.6,120.0,"{'lat': -6.267513, 'lng': 106.824107}",-6.267513,106.824107,museum di tengah kebun adalah buah museum simp...
257,258,Museum Gedung Sate,Salah satu museum yang menjadi ikon kota ini a...,Budaya,Bandung,5000,4.6,NaN,"{'lat': -6.902637899999999, 'lng': 107.619169}",-6.902638,107.619169,salah satu museum yang jadi ikon kota ini adal...
254,255,Museum Barli,Museum Barli adalah sebuah museum yang terleta...,Budaya,Bandung,15000,4.4,180.0,"{'lat': -6.878238699999999, 'lng': 107.5875349}",-6.878239,107.587535,museum barli adalah buah museum yang letak di ...
38,39,Museum Macan (Modern and Contemporary Art in N...,Museum Seni Modern dan Kontemporer di Nusantar...,Budaya,Jakarta,100000,4.5,120.0,"{'lat': -6.190942, 'lng': 106.767622}",-6.190942,106.767622,museum seni modern dan kontemporer di nusantar...
20,21,Museum Wayang,Museum Wayang adalah sebuah museum yang berlok...,Budaya,Jakarta,5000,4.5,150.0,"{'lat': -6.1349072, 'lng': 106.8124452}",-6.134907,106.812445,museum wayang adalah buah museum yang lokasi d...
